In [1]:
import os

db_file = os.path.abspath("db.json")

In [2]:
from pymatgen import Structure

struct_si = Structure([[5.468344455957462, 0.0, -0.0], 
                       [-0.0, 5.468344455957462, 0.0], 
                       [0.0, -0.0, 5.468344455957462]], 
                      
                      ["Si"]*8, 
                      
                      [[-0.0, -0.0, 0.5],
                       [0.75, 0.75, 0.75],
                       [0.0, 0.5, -0.0],
                       [0.75, 0.25, 0.25],
                       [0.5, 0.0, -0.0],
                       [0.25, 0.75, 0.25],
                       [0.5, 0.5, 0.5],
                       [0.25, 0.25, 0.75]])

In [3]:
from atomate.vasp.workflows import wf_structure_optimization

c = {"DB_FILE": db_file}
wf = wf_structure_optimization(struct_si, c)

In [4]:
from atomate.vasp.workflows import get_wf_elastic_constant

wf_elastic = get_wf_elastic_constant(struct_si, db_file=db_file, 
                              stencils=[[0.01], [0.01], [0.01], [0.03], [0.03], [0.03]], 
                              copy_vasp_outputs=True)

In [5]:
wf.append_wf(wf_elastic, wf.leaf_fw_ids)

{-8, -7, -6, -5, -4, -3, -2}

In [6]:
from atomate.vasp.powerups import add_modify_incar


wf = add_modify_incar(wf, {'incar_update': {'EDIFF': 1e-6, 'ENCUT': 700}})

In [7]:
from helper_functions import elastic_simulator

wf = elastic_simulator(wf)

In [8]:
from fireworks.utilities.fw_utilities import plot_wf

plot_wf(wf, depth_factor=2, numerical_label=True)

In [9]:
from fireworks import LaunchPad

lp = LaunchPad()
lp.reset("", require_password=False)

2017-08-05 14:51:10,734 INFO Performing db tune-up
2017-08-05 14:51:10,749 INFO LaunchPad was RESET.


In [10]:
lp.add_wf(wf)

2017-08-05 14:51:10,839 INFO Added a workflow. id_map: {-8: 1, -7: 2, -6: 3, -5: 4, -4: 5, -3: 6, -2: 7, -1: 8}


{-8: 1, -7: 2, -6: 3, -5: 4, -4: 5, -3: 6, -2: 7, -1: 8}

In [11]:
from fireworks import FWorker
from fireworks.core.rocket_launcher import rapidfire

rapidfire(lp)

2017-08-05 14:51:10,901 INFO Created new dir /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-10-901393
2017-08-05 14:51:10,902 INFO Launching Rocket
2017-08-05 14:51:10,933 INFO RUNNING fw_id: 8 in directory: /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-10-901393
2017-08-05 14:51:10,947 INFO Task started: FileWriteTask.
2017-08-05 14:51:10,948 INFO Task completed: FileWriteTask 
2017-08-05 14:51:10,953 INFO Task started: {{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}.
2017-08-05 14:51:10,965 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}} 
2017-08-05 14:51:10,968 INFO Task started: {{atomate.vasp.firetasks.write_inputs.ModifyIncar}}.
2017-08-05 14:51:10,972 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.ModifyIncar}} 
2017-08-05 14:51:10,976 INFO Task started: {{atomate.vasp.firetasks.run_calc.RunVaspFake}}.
2017-08-05 14:51:10,997 INFO atomate.vasp.firetasks.run_calc RunVaspF

/home/km/Software/matproj/pymatgen/pymatgen/io/vasp/outputs.py:822: UserWarning: No POTCAR file with matching TITEL fields was found in /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-11-366218
  " was found in {}".format(os.path.abspath(p)))


2017-08-05 14:51:12,134 INFO Created new dir /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-12-133953
2017-08-05 14:51:12,138 INFO Launching Rocket
2017-08-05 14:51:12,176 INFO RUNNING fw_id: 6 in directory: /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-12-133953
2017-08-05 14:51:12,190 INFO Task started: {{atomate.vasp.firetasks.glue_tasks.CopyVaspOutputs}}.
2017-08-05 14:51:12,436 INFO Task completed: {{atomate.vasp.firetasks.glue_tasks.CopyVaspOutputs}} 
2017-08-05 14:51:12,441 INFO Task started: {{atomate.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}}.
2017-08-05 14:51:12,457 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}} 
2017-08-05 14:51:12,461 INFO Task started: {{atomate.vasp.firetasks.write_inputs.ModifyIncar}}.
2017-08-05 14:51:12,464 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.ModifyIncar}} 
2017-08-05 14:51:12,472 INFO Task started: {{atomate.vasp.f

/home/km/Software/matproj/pymatgen/pymatgen/io/vasp/outputs.py:822: UserWarning: No POTCAR file with matching TITEL fields was found in /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-12-133953
  " was found in {}".format(os.path.abspath(p)))


2017-08-05 14:51:12,907 INFO RUNNING fw_id: 5 in directory: /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-12-867088
2017-08-05 14:51:12,919 INFO Task started: {{atomate.vasp.firetasks.glue_tasks.CopyVaspOutputs}}.
2017-08-05 14:51:13,172 INFO Task completed: {{atomate.vasp.firetasks.glue_tasks.CopyVaspOutputs}} 
2017-08-05 14:51:13,175 INFO Task started: {{atomate.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}}.
2017-08-05 14:51:13,191 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}} 
2017-08-05 14:51:13,194 INFO Task started: {{atomate.vasp.firetasks.write_inputs.ModifyIncar}}.
2017-08-05 14:51:13,197 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.ModifyIncar}} 
2017-08-05 14:51:13,200 INFO Task started: {{atomate.vasp.firetasks.run_calc.RunVaspFake}}.
2017-08-05 14:51:13,222 INFO atomate.vasp.firetasks.run_calc RunVaspFake: verified inputs successfully
2017-08-05 14:51:13,226 INFO atomate.va

...

2017-08-05 14:51:14,382 INFO Task completed: {{atomate.common.firetasks.glue_tasks.PassResult}} 
2017-08-05 14:51:14,416 INFO Rocket finished
2017-08-05 14:51:14,424 INFO Created new dir /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-14-424681
2017-08-05 14:51:14,425 INFO Launching Rocket
2017-08-05 14:51:14,471 INFO RUNNING fw_id: 3 in directory: /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-14-424681
2017-08-05 14:51:14,486 INFO Task started: {{atomate.vasp.firetasks.glue_tasks.CopyVaspOutputs}}.
2017-08-05 14:51:14,735 INFO Task completed: {{atomate.vasp.firetasks.glue_tasks.CopyVaspOutputs}} 
2017-08-05 14:51:14,738 INFO Task started: {{atomate.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}}.
2017-08-05 14:51:14,755 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}} 
2017-08-05 14:51:14,759 INFO Task started: {{atomate.vasp.firetasks.write_inputs.ModifyIncar}}.
2017-08-05 14:5

/home/km/Software/matproj/pymatgen/pymatgen/io/vasp/outputs.py:822: UserWarning: No POTCAR file with matching TITEL fields was found in /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-14-424681
  " was found in {}".format(os.path.abspath(p)))


2017-08-05 14:51:15,301 INFO Task completed: {{atomate.common.firetasks.glue_tasks.PassResult}} 
2017-08-05 14:51:15,338 INFO Rocket finished
2017-08-05 14:51:15,348 INFO Created new dir /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-15-348259
2017-08-05 14:51:15,351 INFO Launching Rocket
2017-08-05 14:51:15,386 INFO RUNNING fw_id: 2 in directory: /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-15-348259
2017-08-05 14:51:15,398 INFO Task started: {{atomate.vasp.firetasks.glue_tasks.CopyVaspOutputs}}.
2017-08-05 14:51:15,646 INFO Task completed: {{atomate.vasp.firetasks.glue_tasks.CopyVaspOutputs}} 
2017-08-05 14:51:15,650 INFO Task started: {{atomate.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}}.
2017-08-05 14:51:15,665 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}} 
2017-08-05 14:51:15,669 INFO Task started: {{atomate.vasp.firetasks.write_inputs.ModifyIncar}}.
2017-08-05 14:5

/home/km/Software/matproj/pymatgen/pymatgen/io/vasp/outputs.py:822: UserWarning: No POTCAR file with matching TITEL fields was found in /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-15-348259
  " was found in {}".format(os.path.abspath(p)))


2017-08-05 14:51:16,142 INFO Task completed: {{atomate.common.firetasks.glue_tasks.PassResult}} 
2017-08-05 14:51:16,178 INFO Rocket finished
2017-08-05 14:51:16,184 INFO Created new dir /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-16-184641
2017-08-05 14:51:16,185 INFO Launching Rocket
2017-08-05 14:51:16,214 INFO RUNNING fw_id: 1 in directory: /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-16-184641
2017-08-05 14:51:16,225 INFO Task started: {{atomate.vasp.firetasks.parse_outputs.ElasticTensorToDb}}.
2017-08-05 14:51:16,227 INFO atomate.vasp.firetasks.parse_outputs Parsing initial optimization directory: /home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-10-901393
2017-08-05 14:51:16,228 INFO atomate.vasp.drones Getting task doc for base dir :/home/km/Software/matproj/workshop-2017/Atomate/launcher_2017-08-05-21-51-10-901393
2017-08-05 14:51:16,356 INFO atomate.vasp.drones Post-processing dir:/home/km/Soft

In [12]:
from atomate.vasp.database import VaspCalcDb

vasp_db = VaspCalcDb()

In [13]:
from helper_functions import get_task_id

task_id = get_task_id(lp, "elastic deformation 4")
d = vasp_db.collection.find_one({"task_label":"elastic deformation 4"})

In [14]:
d["calcs_reversed"][0]["output"]["ionic_steps"][-1]["stress"]

[[3.25702976, 0.0, -45.03552717],
 [0.0, -0.60369959, 0.0],
 [-45.03552718, 0.0, 1.66978914]]

In [15]:
d = vasp_db.db.elasticity.find_one()

In [16]:
d['elastic_tensor']['ieee_format']

[[146.25385837056845,
  52.26244897838195,
  52.262448978381975,
  0.0,
  -9.25185853854297e-18,
  0.0],
 [52.26244897838195,
  146.25385837056848,
  52.26244897838197,
  0.0,
  0.0,
  -4.625929269271485e-18],
 [52.262448978381975,
  52.26244897838197,
  146.25385837056845,
  -4.625929269271485e-18,
  -2.3129646346357427e-18,
  0.0],
 [0.0, 0.0, -4.625929269271485e-18, 75.22653077258354, 0.0, 0.0],
 [-9.25185853854297e-18,
  0.0,
  -2.3129646346357427e-18,
  0.0,
  75.22653077258354,
  0.0],
 [0.0, -4.625929269271485e-18, 0.0, 0.0, 0.0, 75.22653077258353]]